In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import time
from time import sleep
from datetime import datetime as dt
from datetime import timezone
from datetime import timedelta
import random

In [2]:
from selenium.webdriver.common.keys import Keys

In [3]:
driver = webdriver.Chrome()
# driver.get("file:///Users/bgu/Dropbox/MIT/mass%20tech/scraping/test.htm");

In [4]:
class Confession:
    
    def __init__(self, fb_id, timestamp, n_reacts, post_number, full_text):
        self.fb_id = fb_id
        self.timestamp = timestamp
        self.n_reacts = n_reacts
        self.post_number = post_number
        self.full_text = full_text
        
    def __str__(self):
        return self.fb_id + "\n" + str(self.timestamp) + "\n" + "reactions: " + str(self.n_reacts) + "\n" + self.post_number + "\n" + str(self.full_text)
        
confessions = []

In [5]:
def get_full_text(userContentWrapper):
    seeMoreElement = None
    continueReadingElement = None
    try:
        seeMoreElement = userContentWrapper.find_element_by_class_name("see_more_link")
        ActionChains(driver).move_to_element(seeMoreElement).click(seeMoreElement).perform()
    except:
        pass
    try:
        contentElement = userContentWrapper.find_element_by_class_name("userContent")
        return list(map(lambda element: element.text, contentElement.find_elements_by_tag_name("p")))
    except:
        print("couldn't find post")
        return ["couldn't find post"]
    
def get_date_time(userContentWrapper):
    try:
        abbr_elements = userContentWrapper.find_elements_by_tag_name("abbr")
        seconds_since_epoch = None
        for abbr_element in abbr_elements:
            # check if has a child element with class timestampContent
            try:
                abbr_element.find_element_by_class_name("timestampContent")
                seconds_since_epoch = abbr_element.get_attribute("data-utime")
                return dt.fromtimestamp(float(seconds_since_epoch), tz=timezone(timedelta(hours=-5)))
            except:
                pass
    except:
        print("no timestamp found")
        return None
    return None

def get_post_id(userContentWrapper):
    try:
        link_element = userContentWrapper.find_element_by_class_name("_5pcq")
        link_text = link_element.get_attribute("href")
        post_id = link_text[link_text.find("posts/") + 6 : link_text.find("?")]
        return post_id
    except:
        print("couldn't get post id")
        return "?"

def get_n_reacts(userContentWrapper):
    n_reacts = 0
    try:
        reacts_text = userContentWrapper.find_element_by_class_name("_4arz").find_element_by_tag_name("span").text
        number_text = "0"
        for c in reacts_text:
            if c.isdigit():
                number_text += c
        n_reacts = int(number_text)
        if " and " in reacts_text:
            n_reacts += 1
        if "," in reacts_text:
            n_reacts += 1
    except:
        print("no reacts, or couldn't parse number of reacts")
        pass
    return n_reacts

In [6]:
SCROLL_PAUSE_TIME = 0.5

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

count = 0
last_timestamp = dt.fromtimestamp(time.time(), tz=timezone(timedelta(hours=-5)))

while last_timestamp > dt.fromtimestamp(1502827742.0, tz=timezone(timedelta(hours=-5))):
    while count < 30:
        print(count)
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        sleep(SCROLL_PAUSE_TIME)
        count += 1

        # Calculate new scroll height and compare with last scroll height
        # new_height = driver.execute_script("return document.body.scrollHeight")
        # if new_height == last_height:
        #     break
        # last_height = new_height
    count = 0
    print("checking again")
    userContentWrapperElements = driver.find_elements_by_class_name("userContentWrapper")
    last_timestamp = get_date_time(userContentWrapperElements[-1])
    print(last_timestamp)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
checking again
2018-06-01 10:49:00-05:00
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
checking again
2018-05-05 17:38:00-05:00
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
checking again
2018-03-13 18:34:00-05:00
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
checking again
2018-02-03 22:36:04-05:00
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
checking again
2018-01-13 17:01:27-05:00
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
checking again
2017-12-27 23:35:00-05:00
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
checking again
2017-12-11 03:50:00-05:00
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
checking again
2017-11-17 00:20:00-05:00
0
1
2
3
4
5
6
7
8
9
10
11
12
13


In [7]:
userContentWrapperElements = driver.find_elements_by_class_name("userContentWrapper")

In [8]:
for userContentWrapperElement in userContentWrapperElements:
    fb_id = get_post_id(userContentWrapperElement)
    timestamp = get_date_time(userContentWrapperElement)
    n_reactions = get_n_reacts(userContentWrapperElement)
    full_text = get_full_text(userContentWrapperElement)
    post_number = full_text[0][:full_text[0].find(" ")] if len(full_text) > 0 else "N/A"
    
    # if timestamp is not None and timestamp < dt.fromtimestamp(1525213602.0, tz=timezone(timedelta(hours=-5))):
        # may 1, 2018
        # print("done")
        # break
    confession = Confession(fb_id, timestamp, n_reactions, post_number, full_text)
    confessions.append(confession)
    print(confession.post_number)
    if random.random() < 0.05:
        print(confession)
        print("\n")
print("done")

no reacts, or couldn't parse number of reacts
GET
#13877
I
#13876
#13875
no reacts, or couldn't parse number of reacts
#13874
no reacts, or couldn't parse number of reacts
#13873
#13872
#13871
#13870
#13869
#13868
#13867
#13866
1947990781936626
2018-09-08 09:45:27-05:00
reactions: 19
#13866
['#13866 As an upperclassman, nothing warms my cold heart like hearing freshman complain and struggle on their literal first pset. The innocence of not knowing how fucked they will be once it gets hard lmao.']


no reacts, or couldn't parse number of reacts
#13865
#13864
#13863
#13862
no reacts, or couldn't parse number of reacts
#13861
no reacts, or couldn't parse number of reacts
#13860
#13859
#13858
#13857
#13857
#13856
#13855
#13854
#13853
#13852
#13851
#13850
#13849
#13848
#13847
1946435778758793
2018-09-07 01:58:06-05:00
reactions: 23
#13847
['#13847 Webkinz is awesome']


#13846
no reacts, or couldn't parse number of reacts
#13845
#13844
#13843
#13842
no reacts, or couldn't parse number of re

#13354
#13342
1760251090710597
2018-04-19 17:43:00-05:00
reactions: 8
#13342
['#13342 I woke up in the middle of the night because of nightmare about mental hospital again.']


#13339
#13338
no reacts, or couldn't parse number of reacts
#13334
#13332
#13331
#13315
#13310
#13297
#13295
1754289661306740
2018-04-13 12:44:00-05:00
reactions: 2
#13295
['#13295 When I do partner projects or group projects, I noticed that guys in my group tend to do most of the work. As a female, it’s easier for me to get out of the work. Does anyone else—girls or guys—also have this experience?']


#13286
no reacts, or couldn't parse number of reacts
#13283
#13275
#13274
#13269
#13267
#13260
#13256
#13249
#13245
#13240
#13239
#13234
#13227
#13225
#13224
#13223
#13222
#13219
#13200
#13203
#13222
#13205
#13199
#13195
#13185
#13176
#13173
#13172
#13166
#13161
#13162
#13160
#13145
#13140
#13136
#13127
#13123
#13115
#13113
#13111
#13107
#13101
no reacts, or couldn't parse number of reacts
#13096
#13100
#13097
#13

#11596
#11591
#11572
#11571
#11565
#11555
#11551
#11550
#11548
#11547
#11545
#11544
Ava
#11538
#11535
#11512
#11509
#11502
#11497
#11496
#11493
#11492
#11485
#11476
#11474
1655370564531984
2018-01-08 23:45:02-05:00
reactions: 36
#11474
['#11474 i want all my friends to know i have a 5.0']


#11442
#11459
no reacts, or couldn't parse number of reacts
#11456
#11447
#11445
#11434
#11440
#11427
#11422
#11413
#11404
#11408
#11407
#11410
#11406
#11399
#11392
#11394
#11380
#11373
#11374
#11376
#11360
#11357
#11354
#11349
#11347
#11341
#11332
#11329
#11321
#11316
#11325
#11314
#11312
#11310
#11309
#11300
#11295
#11294
#11297
#11280
#11284
#11275
#11273
#11268
#11256
#11258
#11246
#11243
#11244
#11239
#11242
#11248
#11251
#11226
#11219
#11224
#11216
#11220
#11225
#11222
#11217
#11223
#11215
#11201
#11177
#11173
#11168
#11191
#11181
#11183
#11192
#11171
#11195
#11197
#11167
#11172
#11189
#11176
#11200
#11137
#11135
#11158
#11142
#11134
#11147
#11148
#11146
#11139
#11122
#11120
#11110
#11121
#110

#9809
#9834
#9777
#9780
#9789
#9772
#9788
#9793
#9795
#9790
#9796
#9762
#9754
#9739
#9744
#9742
#9755
#9743
#9737
#9723
#9725
#9731
#9722
#9727
#9735
#9728
#9694
#9689
#9696
#9701
#9716
#9697
#9713
#9693
#9712
Dear
#9695
#9709
#9705
#9698
#9678
#9664
#9655
#9684
#9676
#9680
#9682
#9650
#9629
#9645
#9637
#9633
#9619
#9648
#9621
#9649
#9625
#9577
#9580
no reacts, or couldn't parse number of reacts
#9590
#9596
#9598
#9586
1587086388027069
2017-11-03 08:00:00-05:00
reactions: 8
#9586
['#9586 Girl-whos-name-I-dont-know in 21M.030 you have the most adorable smile']


#9616
#9601
1587088568026851
2017-11-03 08:00:00-05:00
reactions: 107
#9601
['#9601 Shoutout to 6.031 for turning memes into a pset and killing the only joy I had left in life']


#9597
#9617
#9599
1587088371360204
2017-11-03 08:00:00-05:00
reactions: 9
#9599
['#9599 For the love of christ, could you please shut the fuck up at Hayden library? I go here to study, not to overhear your inane chatter. Presumably there are other plac

n_bad_scrape = 0
for confession in confessions:
    if confession.full_text[0] == "couldn't find post":
        print(confession.full_text)

In [17]:
f = open("confessions.txt", "w")
for confession in confessions:
    print(confession.post_number)
    f.write(str(confession))
    f.write("\n\n")
f.close()

You
#2462
#2461
#2460
#2459
#2458
#2457
#2456
#2455
#2454
#2453
#2452
#2451
#2450
#2449
#2448
#2447
#2446
#2445
#2444
#2443
#2442
#2441
Hello!
#2440
#2439
#2438
#2437
#2436
#2435
#2434
#2433
#2432
#2431
#2430
#2429
#2428
#2427
#2426
#2425
#2424
#2423
#2422
#2421
#2420
#2419
#2418
#2417
#2416
#2415
#2414
#2413
#2412
#2411
#2410
#2409
#2408
#2407
#2406
#2405
#2404
#2403
#2402
#2401
#2400
#2399
#2398
#2397
#2396
#2395
#2394
#2393
#2392
#2391
#2390
#2389
#2388
#2387
#2386
#2385
#2384
#2383
#2382
#2381
#2380
#2379
#2377
#2376
#2375
#2374
#2373
#2372
#2371
#2370
#2369
#2368
#2367
#2366
#2365
#2364
#2363
#2362
#2361
#2360
#2359
#2358
#2357
#2356
#2355
#2354
#2353
#2352
#2351
#2350
#2349
#2348
#2347
#2346
#2345
#2331
#2327
#2338
#2341
#2336
#2334
#2339
#2332
#2330
#2324
#2328
#2337
#2326
#2333
#2343
#2329
#2335
#2340
#2325
#2323
#2322
#2321
#2320
#2319
#2318
#2317
#2316
#2315
#2314
#2313
#2312
#2311
#2310
#2309
#2307
#2306
#2305
#2304
#2303
#2302
#2301
#2300
#2299
#2298
#2297
#2296
#2295
#2294

In [11]:
print(confessions[100].timestamp.year)

2018


In [12]:
confessions_by_month = {}

confessions_by_month["aug17"] = list(filter(lambda confession: confession.timestamp.month == 8 and confession.timestamp.year == 2017, confessions))
confessions_by_month["sep17"] = list(filter(lambda confession: confession.timestamp.month == 9 and confession.timestamp.year == 2017, confessions))
confessions_by_month["oct17"] = list(filter(lambda confession: confession.timestamp.month == 10 and confession.timestamp.year == 2017, confessions))
confessions_by_month["nov17"] = list(filter(lambda confession: confession.timestamp.month == 11 and confession.timestamp.year == 2017, confessions))
confessions_by_month["dec17"] = list(filter(lambda confession: confession.timestamp.month == 12 and confession.timestamp.year == 2017, confessions))
confessions_by_month["jan18"] = list(filter(lambda confession: confession.timestamp.month == 1 and confession.timestamp.year == 2018, confessions))
confessions_by_month["feb18"] = list(filter(lambda confession: confession.timestamp.month == 2 and confession.timestamp.year == 2018, confessions))
confessions_by_month["mar18"] = list(filter(lambda confession: confession.timestamp.month == 3 and confession.timestamp.year == 2018, confessions))
confessions_by_month["apr18"] = list(filter(lambda confession: confession.timestamp.month == 4 and confession.timestamp.year == 2018, confessions))
confessions_by_month["may18"] = list(filter(lambda confession: confession.timestamp.month == 5 and confession.timestamp.year == 2018, confessions))
confessions_by_month["jun18"] = list(filter(lambda confession: confession.timestamp.month == 6 and confession.timestamp.year == 2018, confessions))
confessions_by_month["jul18"] = list(filter(lambda confession: confession.timestamp.month == 7 and confession.timestamp.year == 2018, confessions))
confessions_by_month["aug18"] = list(filter(lambda confession: confession.timestamp.month == 8 and confession.timestamp.year == 2018, confessions))
confessions_by_month["sep18"] = list(filter(lambda confession: confession.timestamp.month == 9 and confession.timestamp.year == 2018, confessions))

In [13]:
for month in ["aug17", "sep17", "oct17", "nov17", "dec17", "jan18", "feb18", "mar18", "apr18", "may18", "jun18", "jul18", "aug18", "sep18"]:
    confessions_by_month[month].sort(key=lambda confession: confession.n_reacts, reverse=True)

In [15]:
str(confessions_by_month["jan18"][20])

"1657392807663093\n2018-01-10 23:54:02-05:00\nreactions: 66\n#11535\n['#11535 USAMO 2017 was so easy.']"

In [16]:
import pickle

In [17]:
f = open('confessions_all', 'wb')
pickle.dump(confessions, f)
f.close()

In [18]:
for month in ["aug17", "sep17", "oct17", "nov17", "dec17", "jan18", "feb18", "mar18", "apr18", "may18", "jun18", "jul18", "aug18", "sep18"]:
    f = open("confessions_" + month + ".txt", "w")
    for confession in confessions_by_month[month]:
        print(confession.post_number)
        f.write(str(confession))
        f.write("\n\n")
    f.close()

#9459
Some
CONFESSIONS
#9469
#9458
#9485
#9484
#9455
#9473
#9456
#9453
#9461
#9507
#9510
#9492
#9503
#9494
#9504
#9543
#10118
#9727
#10055
#9676
#9577
#9680
#10117
#9801
#10190
#9755
#10082
#9601
#9919
#9744
#10037
#9754
#10091
#9695
#10069
#9728
#10047
#10107
#9907
#9848
#9922
#9709
#9568
#9619
#9841
#10207
#9789
#9870
#10053
#9958
#10070
#10174
#9705
#10038
#9735
#9872
#9842
#9999
#9742
#9701
#9598
#10156
#10035
#9527
#10005
#10075
#9649
#9596
#10040
#10162
#10152
#10064
#9915
#9851
#9830
#10177
#9548
#10088
#10113
#10214
#10033
#9731
#9633
#10164
#9550
#10071
#9823
#9616
#9723
#9689
#10191
#9739
#9737
#9693
#9833
#10085
#10072
#9971
#10193
#9539
#9512
#9804
#9561
#9525
#10159
#10135
#10049
#9834
#9684
#9625
#9580
#10163
#10187
#9972
#9790
#9743
#9863
#9648
#9541
#9536
#10180
#10015
#9655
#9947
#9925
#9853
#9780
#9696
#9650
#9518
#9697
#9629
#10114
#9900
#10197
#10194
#10130
#9918
#9856
#9682
#10208
#9818
#9762
#9617
#10222
#9976
#9984
#9874
#9725
#9716
#9982
#9788
#9698
#9621
#10108

#13777
#13775
#13779
#13782
#13787
#13806
#13805
#13798
#13809
#13792
#13797
#13812
#13780
#13774
#13791
#13808
#13811
#13789
#13786
#13778
#13793
#13802
#13773
#13781
#13803
#13799
#13795
#13788
#13784
#13770
#13800
#13796
#13783
#13776
#13826
A
#13816
#13818
#13846
#13832
#13819
#13835
#13877
I
#13870
#13840
#13838
#13847
#13827
#13834
#13857
#13856
#13866
#13843
#13831
#13867
#13869
#13821
#13823
#13854
#13829
#13842
#13844
#13850
#13836
#13828
N/A
#13876
#13863
#13852
#13817
#13813
#13859
#13851
#13848
#13815
#13864
#13857
#13855
#13825
#13814
#13862
#13839
#13830
#13824
#13875
#13872
#13833
#13822
#13868
#13858
#13849
#13820
GET
#13874
#13873
#13871
#13865
#13861
#13860
#13853
#13845
#13841
#13837


In [20]:
print(confessions_by_month["sep17"][0])

1546126668789708
2017-09-19 11:06:00-05:00
reactions: 165
#9459
["#9459 I texted my ex last Friday just to make myself angry. I bench more when I'm angry and I wanted to PR."]
